# Parse and Interpret NMEA data stream from GPS

Brett Deaton - Summer 2021

This notebook translates the raw data stream from a GPS receiver into meaningful data. The raw data stream is read in from the file `nmea-data_stream.csv`, represented in the NMEA-0183 format.

Here's a reference on the NMEA-0183 standard from [navspark](https://navspark.mybigcommerce.com/content/NMEA_Format_v0.1.pdf). We'll use only the Global Positioning System Fix Data line denoted `GNGGA`.

The GPS receiver is the [u-blox SAM-M8Q](https://www.u-blox.com/en/product/sam-m8q-module). You can buy one already attached to a PCB by [sparkfun](https://www.digikey.com/en/products/detail/sparkfun-electronics/GPS-15210/10064422), already optimized with a copper-plan serving as an antenna.

### Set up

In [ ]:
import csv # to quickly read in comma-separated values

In [ ]:
# read fix data from the csv and convert to 2D list of strings
points = []
with open('nmea-data_stream.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        if "GGA" in row[0] and row[6]!="0": # only use $GNGGA rows with fixes
            points.append(row[1:])

In [ ]:
# look at the first point for a visual check
for x in points[0]:
    print(x, end=" ")

### Extract and Convert Data

Some fields we might be interested in, summarizing the NMEA-0183 format:

| index | meaning            | string format
|-------|--------------------|---------------
| 0     | UTC time           | hhmmss.ss
| 1     | latitude           | ddmm.mmmmm
| 3     | longitude          | dddmm.mmmmm
| 8     | altitude (m)       | x.x

In [ ]:
# make a list of timestamps
times = []
for x in points:
    times.append([int(x[0][:2]),    # hr
                  int(x[0][2:4]),   # min
                  float(x[0][4:])]) # sec

In [ ]:
# make a list of latitudes
lats = []
for x in points:
    sign = 1 if x[2]=="N" else -1
    lats.append([sign*int(x[1][:2]), # deg
                 float(x[1][2:])])   # min

In [ ]:
# make a list of longitudes
longs = []
for x in points:
    sign = 1 if x[2]=="E" else -1
    longs.append([sign*int(x[3][:3]), # deg
                  float(x[3][3:])])   # min

In [ ]:
# make a list of altitudes
alts = []
for x in points:
    alts.append(float(x[8])) # altitude

### Todo

Tasks left to complete:
* use pandas or numpy to organize and display dataset
* analyze position accuracy over the recorded data
* look for patterns in GPS quality indicator, i.e why some fixes unavailable
* examine number of satellites used for fix
* compute checksums of each row and compare to recorded checksum